In [ ]:
import pandas as pd
#from datascience import *
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
import itertools
import seaborn as sn
from textblob import TextBlob
import twint
%matplotlib inline
import nltk
from nltk.stem import *
import re 

In [2]:
def preprocessor(data):
    
    # TfidfVectorizer puts all letters in lowercase by default
    data = data.lower()

    # sub hyphens with no space
    data = re.sub("\-", '', data)
    
    # sub non letter & non digit characters w/ a space
    data = re.sub("[^A-Za-z0-9 \\n]","", data)
    
    # sub digit characters with '#' character
    data = re.sub("[\d]", "\#", data) 
    
    # removing stop words
    stop_words = ['and', 'but', 'the', 'just', 'or', 'it', 'of', 'to', 'her','he']
    for word in stop_words:
        data = data.replace(word, "")
    
    # stemming words
    stem = PorterStemmer()
    for word in data:
        data = data.replace(word, stem.stem(word))
        
    # shorten long words of length 7 or more
    data = re.sub('(\w{7})\w+', '\\1', data)
    
    return data

In [3]:
def tvect(data):
    vect = TfidfVectorizer(min_df = 5, ngram_range = (1,4))
    fitted = vect.fit_transform(data)
    return fitted, vect.get_feature_names()

def get_polarity(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

In [ ]:
def featurize(csv, path):
    #read in csv & make a df
    df = pd.read_csv(path+ csv)
    
    df = df.rename(columns={'id': 'text'})
    #df = df.set_index('tweet_id')
    
    #add polarity column
    df['polarity'] = df['full_text'].apply(get_polarity)
    
    #add preprocessed column
    df['preprocess_full_text'] = df.full_text.apply(lambda x: preprocessor(x))
    
    #add feature names from TfidfVectorizer
    featurized = tvect(df.preprocess_full_text)
    df1 = pd.DataFrame(featurized[0].toarray(), columns=featurized[1])
    df = pd.concat([df, df1], axis=1)
    
    
    return df, featurized[1]
